In [ ]:
# 모델 2 - 기능 3에 대해 추가 처리 가능한 모델2 작성 (1)
#### 모델 2는 모델 1에서 기능 3 지하철 트리거 인식 시 작동되는 모델임
#### 모델 1에서 기능 3으로 분류(카테고리 :3) 인 경우 해당 모델에서 역별 분류 작업 수행
#### 최종 수정일 : 20-05-27
* * *

## LSTM 모델

* * *
## 2. 모델링
### 2.1 데이터 구성

> [변경사항]

* * *
> [기존과 동일]
- model_1.npz 불러오기

In [1]:
import glob
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 알림, 차량 엔진, 차량 경적, 지하철 트리거 순
sound_data = np.load('data.npz')
X_train = sound_data['X']
y_train = sound_data['y']
X_train.shape, y_train.shape

((129, 193), (129, 1))

In [3]:
X_train.shape, y_train.shape

((129, 193), (129, 1))

* * *
## 2. 모델링
### 2.2 모델 학습

> [변경사항]
- lstm 사용
- 이전 모델의 경우 파라미터 조정에 초점을 두었으나 ver 3.5에서는 layer 구성에 초점을 두어 진행
* * *
> [기존과 동일]


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

In [5]:
from keras import models
from keras import layers
from keras.layers import *
from keras import optimizers

Using TensorFlow backend.


In [9]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping

K.clear_session()
model = Sequential() # Sequeatial Model
model.add(LSTM(20, input_shape=(193, 1))) # (timestep, feature)
model.add(Dense(1)) # output = 1
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [10]:
#X_train = X_train.values
X_train = X_train.reshape(X_train.shape[0], 193, 1)

In [13]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(X_train, y_train, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop])

Epoch 1/100
129/129 [==============================] - 0s 3ms/step - loss: 157.3219
Epoch 2/100
129/129 [==============================] - 0s 3ms/step - loss: 157.1995
Epoch 3/100
129/129 [==============================] - 0s 3ms/step - loss: 157.1315
Epoch 4/100
129/129 [==============================] - 0s 2ms/step - loss: 156.9902
Epoch 5/100
129/129 [==============================] - 0s 3ms/step - loss: 156.9106
Epoch 6/100
129/129 [==============================] - 0s 3ms/step - loss: 156.7834
Epoch 7/100
129/129 [==============================] - 0s 2ms/step - loss: 156.7113
Epoch 8/100
129/129 [==============================] - 0s 3ms/step - loss: 156.6118
Epoch 9/100
129/129 [==============================] - 0s 2ms/step - loss: 156.5275
Epoch 10/100
129/129 [==============================] - 0s 2ms/step - loss: 156.4403
Epoch 11/100
129/129 [==============================] - 0s 2ms/step - loss: 156.3570
Epoch 12/100
129/129 [==============================] - 0s 2ms/step - loss

* * *
## 2. 모델링
### 2.3 모델 저장

> [변경사항]
- pkl, json, pb, tflite로 저장
* * *
> [기존과 동일]


In [14]:
# 모델 pkl로 저장하기
import joblib
joblib.dump(model, 'model_2.pkl')

['model_2.pkl']

In [15]:
# 모델 json으로 저장하기
model_2 = model.to_json()
# model = model_from_json(json_string)

In [16]:
# 모델 h5로 저장하기
from keras.models import load_model
model.save('model_2')
model.save('model_2.h5')

In [18]:
# 모델 pb로 저장하기
model = keras.models.load_model('model_2', compile=False)
model.save('model/pb/',save_format=tf)

INFO:tensorflow:Assets written to: model/pb/assets


In [19]:
#모델 tflite 로 저장하기
saved_model_dir='model/pb/'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS,
                                     tf.lite.OpsSet.SELECT_TF_OPS]
tfilte_mode=converter.convert()
open('model/tflite/model_1.tflite','wb').write(ftlite_model)

ConverterError: See console for info.
2020-05-27 21:43:09.825377: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-05-27 21:43:09.825766: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-05-27 21:43:14.295202: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2020-05-27 21:43:14.296981: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2020-05-27 21:43:14.324665: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce 940MX computeCapability: 5.0
coreClock: 1.2415GHz coreCount: 3 deviceMemorySize: 2.00GiB deviceMemoryBandwidth: 14.92GiB/s
2020-05-27 21:43:14.326032: W tensorfl